In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import sys
sys.path.append('./helpers')
from py_fp import tidy_tdt_extract_and_tidy

dir_raw = r'D:\non_2p\Brandy\raw'
dir_extracted = r'D:\non_2p\Brandy\extracted'

tidy_tdt_extract_and_tidy(dir_raw, dir_extracted)

In [ ]:
fname = 'hcp_37_B_11092022'
session_info_path = os.path.join(dir_extracted, f'{fname}_info.feather')

data_path = os.path.join(dir_extracted, f'{fname}_streams_data.feather')
data_info_path = os.path.join(dir_extracted, f'{fname}_streams_info.feather')

epoc_data_path = os.path.join(dir_extracted, f'{fname}_epocs_data.feather')
epoc_info_path = os.path.join(dir_extracted, f'{fname}_epocs_info.feather')

In [ ]:
session_info = pd.read_feather(session_info_path)

## work on behavioral data

In [ ]:
epoc_data = pd.read_feather(epoc_data_path)
epoc_data.head()

In [ ]:
epoc_data['name'].unique() # grab unique condition names

In [ ]:
# grab event times for condition 1
event_one = epoc_data['onset'][epoc_data['name'] == 'PC1/'].values

## Work on tseries data

In [ ]:
# grab event times for all conditions via for loop
event_times_dict = {}
for event_name in epoc_data['name'].unique():
    event_times_dict[event_name] = epoc_data['onset'][epoc_data['name'] == event_name].values

In [ ]:
data = pd.read_feather(data_path)
data_info = pd.read_feather(data_info_path)
fs = data_info['fs'][0] # assuming sampling rate is the same across channels

data_405 = data[data['channel']=='_405A']['raw_au'].values
data_465 = data[data['channel']=='_465A']['raw_au'].values

tvec = np.linspace(0,len(data_405)/fs, len(data_405))

data

In [ ]:
# GuPPY version
# function to fit control channel to signal channel
def controlFit(control, signal):
    
    p = np.polyfit(control, signal, 1)
    arr = (p[0]*control)+p[1]
    return arr

# function to compute deltaF/F using fitted control channel and filtered signal channel
def deltaFF(signal, control):
    
    res = np.subtract(signal, control)
    normData = np.divide(res, control)
    #deltaFF = normData
    normData = normData*100

    return normData

fitted_control = controlFit(data_405, data_465)
corrected_data = deltaFF(data_465, fitted_control)

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(tvec, data_405)
plt.plot(tvec, data_465)
plt.plot(tvec, fitted_control)
plt.plot(tvec, corrected_data)
plt.xlabel('Time (s)')
plt.ylabel('Fluorescence')
#plt.xlim([0.5, tvec[-1]]) # test: how to get rid of artifact
plt.legend(['405', '470', 'fit curve', 'corrected']);

### Convert behavioral event times to samples: to prepare for indexing and snipping out trial activity windows.

#### Requires knowledge on: dictionaries and functions 

The data itself is not characterized by units of time, rather each item in the vector is a "sample" in the recording and does not hold any information on timing

On the other hand our behavioral events are in units of time. Accordingly, to extract trial snippits from the photometry data, we need to figure out which sample corresponds to a given event time

We can use this line of code : `np.argmin(abs(tvec - time))`

In [ ]:
print(f'First event from condition one: {event_one[0]}')

print(f"The sample that corresponds to {event_one[0]} is {np.argmin(abs(tvec - event_one[0]))}")

print(f"Sample {np.argmin(abs(tvec - event_one[0]))}'s time: {tvec[330439]}")

### Let's extract the activity trace from one trial for demonstration purposes

In [ ]:
# initialize start/end times for trial window
event_window_time = np.array([-1, 10])
event_window_samples = (event_window_time*fs).astype(int) # samples are integers

# generate vector the maps time onto samples in the event trial
trial_window_num_samples = event_window_samples[1] - event_window_samples[0]
tvec_trial = np.linspace(event_window_time[0], event_window_time[1], trial_window_num_samples)

# generate vector of every sample between start and end samples
# then offset to specific event's window by adding the event sample
trial_template_indices = np.arange(event_window_samples[0], event_window_samples[1])
trial_samples = trial_template_indices + np.argmin(abs(tvec - event_one[0]))

plt.plot(tvec_trial, corrected_data[trial_samples])

### To analyze all trials, let's do some organization of the event conditions and trial times

In [ ]:
# turn the above code into a function
def get_tvec_sample(tvec, time):
    return np.argmin(abs(tvec - time))

event_samples_dict = {}
for condition_name in epoc_data['name'].unique(): # loop through conditions
    
    print(condition_name)
    tmp_list = []
    
    # go through each event and compute the sample/frame that it occurred on (b/c the list is currently in seconds)
    for event in event_times_dict[condition_name]: # loop through events
        tmp_list.append(get_tvec_sample(tvec, event))
        
    # once we go through converting each time to sample, and add to a list iteratively,
    # store that list to its corresponding condition in the dictionary
    event_samples_dict[condition_name] = tmp_list

### Let's analyze data from all trials

In [ ]:
# bonus explanation
def remove_trials_out_of_bounds(data_end, frame_events, start_samp, end_samp):

    # list of booleans that indicates for each trial if it stays within the start/end samples of the entire recording
    after_start_bool = (frame_events + start_samp) > start_samp # filters out (set to false) trials that would index before the first sample
    before_end_bool = (frame_events + end_samp) < data_end # filters out trials that would index passed the last sample

    """
    One pythonic way to combine the info above. Multiplying the lists generates a union of the two lists.
    Then the if statement will allow trials that are True (falls within the entire recording) through
    """
    keep_events = []
    for idx, item in enumerate(after_start_bool*before_end_bool): 
        if item:
            keep_events.append(frame_events[idx])

    return np.array(keep_events)

In [ ]:
# remove trials that have windows outside of entire recording, then extract data from each trial

data_trial_dict = {}
for condition_name in epoc_data['name'].unique(): # loop through conditions
    
    # use function above to remove trials where the trial window would extend outside of indexable data
    event_samples_dict[condition_name] = remove_trials_out_of_bounds(len(tvec), 
                                                                     event_samples_dict[condition_name], 
                                                                     event_window_samples[0], 
                                                                     event_window_samples[-1])
    
    # initialize numpy array to populate with trial-extracted data
    tmp_trial_array = np.empty([len(event_samples_dict[condition_name]), trial_window_num_samples])
    
    # use trial index template, offset with event sample, and extract trial data
    for idx, event_sample in enumerate(event_samples_dict[condition_name]): # loop through events
        tmp_trial_array[idx,:] = corrected_data[trial_template_indices + event_sample]
    
    # once data are fully populated, add to dictionary
    data_trial_dict[condition_name] = tmp_trial_array

In [ ]:
plt.imshow(data_trial_dict['PC1/'], aspect='auto')
plt.ylabel('Fluorescence')
plt.xlabel('Sample')

In [ ]:
plt.plot(tvec_trial, np.mean(data_trial_dict['Tick'], axis=0))
plt.plot(tvec_trial, np.mean(data_trial_dict['PC1/'], axis=0))
plt.ylabel('Fluorescence')
plt.xlabel('Time')